In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class  DataPreprocessConfig:
    root_dir:Path
    input_preprocess:Path
    image_input:Path
    text_input:Path
    output_preprocess:Path
    unzip_dir:Path



In [4]:
from Image2Recipie.constants import *
from Image2Recipie.utils.common import read_yaml,create_directories

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bisht\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

    def get_data_preprocessing_config(self)->DataPreprocessConfig:
        config=self.config
        create_directories([config.preprocessing.root_dir])
        create_directories([config.preprocessing.input_preprocess])
        create_directories([config.preprocessing.output_preprocess])
        data_preprocess_config=DataPreprocessConfig(
            root_dir=config.preprocessing.root_dir,
            input_preprocess=config.preprocessing.input_preprocess,
            image_input=config.preprocessing.image_input,
            text_input=config.preprocessing.text_input,
            output_preprocess=config.preprocessing.output_preprocess,
            unzip_dir=config.data_ingestion.unzip_dir
        )
        return data_preprocess_config

In [6]:
import os
import urllib.request as request 
import gdown
from Image2Recipie import logger
import tensorflow as tf
from Image2Recipie.utils.common import save_object,get_maxlen,clean_ingredients,CustomTokenizer
import cv2 as cv
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
class DataPreprocess:
    def __init__(self,config:DataPreprocessConfig):
        self.config = config
        self.input_image=None 
    def input_preprocess(self):
        self.data=pd.read_csv(os.path.join(self.config.unzip_dir,'cuisine_updated.csv'))
        
        X=[]
        for i in self.data['image_path']:
            
            img = cv.imread(os.path.join(self.config.unzip_dir, "data/data/" + i))
            img = cv.resize(img, (224, 224))
            img=cv.resize(img,(180,180))
            X.append(img)
        input_image=np.array(X)
        save_object(self.config.image_input,input_image)
        maxlen=get_maxlen(self.data['name'])
        token=Tokenizer()
        token.fit_on_texts(self.data['name'])
        Xtoken=token.texts_to_sequences(self.data['name'])
        Xtoken=pad_sequences(Xtoken,maxlen=maxlen)
        save_object(self.config.text_input,Xtoken)
        save_object(os.path.join(self.config.input_preprocess,'input_tokenizer.pkl'),token)
        logger.info(f'The Preprocessed input is saved in {self.config.input_preprocess}')

    def output_preprcess(self):
        self.data['cleaned_ingredient']=self.data['cleaned_ingredient'].apply(clean_ingredients)
        self.data['cleaned_ingredient']=self.data['cleaned_ingredient'].apply(lambda x:','.join([x.strip() for x in x.split(',')]))
        tokenizer=CustomTokenizer()
        y_tokenized=tokenizer.fit_transform(self.data['cleaned_ingredient'])
        save_object(os.path.join(self.config.output_preprocess,'y_tokenized.pkl'),y_tokenized)
        save_object(os.path.join(self.config.output_preprocess,'tokenizer.pkl'),tokenizer)
        logger.info(f'The Preprocessed output is saved in {self.config.output_preprocess}')



        
    

            
            





In [17]:
try:
    config=ConfigurationManager()
    data_preprocessing_config=config.get_data_preprocessing_config()
    data_preprocessing=DataPreprocess(config=data_preprocessing_config)
    data_preprocessing.input_preprocess()
    data_preprocessing.output_preprcess()
    
except Exception as e:
    raise e

[2024-10-26 14:17:30,465:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-10-26 14:17:30,467:INFO:common:yaml file: params.yaml loaded successfully]
[2024-10-26 14:17:30,469:INFO:common:created directory at: artifacts/preprocessing]
[2024-10-26 14:17:30,470:INFO:common:created directory at: artifacts/preprocessing/inputs_preprocess]
[2024-10-26 14:17:30,471:INFO:common:created directory at: artifacts/preprocessing/output_preprocess]
[2024-10-26 14:18:24,121:INFO:131008179:The Preprocessed input is saved in artifacts/preprocessing/inputs_preprocess]
[2024-10-26 14:18:25,179:INFO:131008179:The Preprocessed output is saved in artifacts/preprocessing/output_preprocess]


In [11]:
df[df['cleaned_ingredient'].isnull()]

,Unnamed: 0,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,image_available,cleaned_ingredient,image_path
2661,2661,Urulaikizhangu Puli Thokku,https://www.archanaskitchen.com/images/archana...,Urulaikizhangu Puli Thokku Recipe (South India...,NaN,NaN,NaN,NaN,\n,To begin making Urulaikizhangu Puli Thokku Rec...,1,NaN,3150.Urulaikizhangu_Puli_Thokku_Recipe_South_I...
2662,2662,Classic Pavakkai Stir Fry,https://www.archanaskitchen.com/images/archana...,Bitter Gourd is known as Pavakkai in Tamil. Ma...,NaN,NaN,NaN,NaN,\n,"To begin making the Classic Pavakkai Stir Fry,...",1,NaN,3151.Pavakkai_Fry_Recipe_Bitter_Gourd_Fry.jpg


'\n'